In [1]:
import pymc as pm
import numpy as np
import pytensor
import pytensor.tensor as pt
import pandas as pd
import arviz as az
import pymc.math as math
import matplotlib.pyplot as plt
import scipy.stats as stats

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [15]:
nTrials = 9600
nUnits = 35
nConditions = 12
nPara = 6
trials_p_block = 50
nAverage = 1000
nRandom = 100
nBlocks = 32
a_max = 0.5
nConditions = 6

bias=np.tile(0,nAverage)
bias_update=0.02

w_min=-1
w_max=1
weight=np.tile(np.array([-45,-30,-15,0,15,30,45])/30,(nAverage,5))

In [3]:
activation = pd.read_csv('Cache.csv', header=None).values.reshape((nConditions, nRandom, nUnits))
df = pd.read_csv('Data_withfeedback.csv')[:9600]
condition = df['Condition'].values-1
correct = df['Correct'].values
contrast = df['Contrast'].values
congruent = df['Congruent'].values

sign_arr = [-1,1,-1,1,-1,1,-1,1,-1,1,-1,1]

li = np.intersect1d(np.where(contrast==0.10),np.where(congruent==0))
mi = np.intersect1d(np.where(contrast==0.15),np.where(congruent==0))
hi = np.intersect1d(np.where(contrast==0.23),np.where(congruent==0))
lc = np.intersect1d(np.where(contrast==0.10),np.where(congruent==1))
mc = np.intersect1d(np.where(contrast==0.15),np.where(congruent==1))
hc = np.intersect1d(np.where(contrast==0.23),np.where(congruent==1))

noise_d =  np.random.normal(0, 1, size=(nTrials,nAverage))
noise_r =  np.random.normal(0, 1, size=(nTrials,nAverage,nUnits))
random_seq = np.random.randint(0,100,(nTrials,nAverage))

sign = np.zeros(nTrials)
for i in range(nTrials):
    sign[i] = sign_arr[condition[i]]

acti = np.zeros((nTrials, nAverage, nUnits))
for i in range(nTrials):
    acti[i] = activation[condition[i]][random_seq[i]]


In [4]:
Acti = pt.tensor3('Acti')
Noise_d = pt.matrix('Noise_d')
Noise_r = pt.tensor3('Noise_r')
Sign = pt.vector('Sign')
Bias = pt.vector('Bias')
Weight = pt.matrix('Weight')
Lr = pt.scalar('Lr')
Bias_strength = pt.scalar('Bias_strength')
Gamma = pt.scalar('Gamma')
A_max = pt.scalar('A_max')
W_max = pt.scalar('W_max')
W_min = pt.scalar('W_min')
Bias_update = pt.scalar('Bias_update')

'''
def update_decisions(Acti, Noise_d, Noise_r, Sign, Bias, Weight, Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update):
    # calculate sign of decision
    Acti_noise = Acti + Noise_r
    Acti_normalized =((1-pt.exp(-Gamma*pt.maximum(Acti_noise,0)))/(1+pt.exp(-Gamma*pt.maximum(Acti_noise,0))))*A_max
    Deci = pt.sign(pt.sum(Acti_normalized* Weight, axis=1) - Bias_strength * Bias + Noise_d)
    # update bias and weight
    Bias= Bias_update* Deci + (1 - Bias_update) * Bias
    Delta= Acti_normalized * Lr * Sign * A_max
    Weight = Weight + (W_max - Weight) * pt.maximum(Delta,0) + (Weight - W_min) * pt.minimum(Delta, 0)
    return Bias, Weight, Deci 

results, updates = pytensor.scan(fn=update_decisions,
                            sequences = [Acti, Noise_d, Noise_r, Sign],
                            outputs_info=[Bias, Weight, None],
                            non_sequences=[Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update])
final_result = results[2]
decision = pytensor.function(inputs=[Acti, Noise_d, Noise_r, Sign, Bias, Weight, Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update], outputs= final_result, updates= updates)
'''


def update_decisions(Acti, Noise_d, Noise_r, Sign, Bias, Weight, Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update):
    # calculate sign of decision
    Acti_noise = Acti + Noise_r
    Acti_normalized =((1-pt.exp(-Gamma*pt.maximum(Acti_noise,0)))/(1+pt.exp(-Gamma*pt.maximum(Acti_noise,0))))*A_max
    Deci = pt.sign(pt.sum(Acti_normalized* Weight, axis=1) - Bias_strength * Bias + Noise_d)
    # update bias and weight
    Bias= Bias_update* Deci + (1 - Bias_update) * Bias
    Delta= Acti_normalized * Lr * Sign * A_max
    Weight = Weight + (W_max - Weight) * pt.maximum(Delta,0) + (Weight - W_min) * pt.minimum(Delta, 0)
    return Bias, Weight, Deci 

results, updates = pytensor.scan(fn=update_decisions,
                            sequences = [Acti, Noise_d, Noise_r, Sign],
                            outputs_info=[Bias, Weight, None],
                            non_sequences=[Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update])
final_result = results[2]
decision = pytensor.function(inputs=[Acti, Noise_d, Noise_r, Sign, Bias, Weight, Lr, Bias_strength, Gamma, A_max, W_max, W_min, Bias_update], outputs= final_result, updates= updates)

In [5]:
lr_range = [0.0005,0.0010,0.0015,0.0020,0.0025,0.0035]
bias_range = [1.0,1.5,2.0,2.5,3.0,3.5]
gamma_range = [0.3,0.6,0.9,1.2,1.5,1.8]
noise_d_range = [0.05,0.10,0.15,0.20,0.25,0.30]
noise_r_range = [0.05,0.10,0.15,0.20,0.25,0.30]
w_init_range = [0.10,0.15,0.20,0.25,0.30,0.35]

search_grid = np.array(np.meshgrid(lr_range, bias_range, gamma_range, noise_d_range, noise_r_range, w_init_range,indexing='xy')).T.reshape(-1, nPara)

In [6]:
for i in range(11):
    para = search_grid[i]
    group_decision = decision(acti, noise_d * para[3], noise_r * para[4], sign, bias, weight * para[5] , para[0], para[1], para[2], a_max, w_max, w_min, bias_update)
    prob = (((group_decision.sum(axis = 1) / nAverage) * sign) + 1) / 2
    block_prob = np.sum(prob[[li,mi,hi,lc,mc,hc]].reshape((6,32,50)), axis=2)/trials_p_block
    print(i)

0


KeyboardInterrupt: 

In [16]:
iSubject = 0
df = pd.read_csv('Data_withfeedback.csv')[iSubject * nTrials : (iSubject + 1) * nTrials]
correct = df['Correct'].values
contrast = df['Contrast'].values
congruent = df['Congruent'].values

li = np.intersect1d(np.where(contrast==0.10),np.where(congruent==0))
mi = np.intersect1d(np.where(contrast==0.15),np.where(congruent==0))
hi = np.intersect1d(np.where(contrast==0.23),np.where(congruent==0))
lc = np.intersect1d(np.where(contrast==0.10),np.where(congruent==1))
mc = np.intersect1d(np.where(contrast==0.15),np.where(congruent==1))
hc = np.intersect1d(np.where(contrast==0.23),np.where(congruent==1))

learning_curve = correct[np.stack([li, mi, hi, lc, mc, hc])].reshape(nConditions,nBlocks,trials_p_block).sum(axis=2)